In [ ]:
#!ollama run gemma3

⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ >>> Send a message (/? for help)

OSError: [Errno 5] Input/output error

In [ ]:
import pandas as pd
import glob
import os
import re
import difflib

input_dir = "data"
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

csv_files = glob.glob(os.path.join(input_dir, "*.csv"))
if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {input_dir}")

dfs = []
for file in csv_files:
    df_temp = pd.read_csv(file)
    df_temp["SourceFile"] = os.path.basename(file)
    dfs.append(df_temp)

df = pd.concat(dfs, ignore_index=True)

df_text = (df['Title'].fillna('') + ' ' + df['Text'].fillna('')).str.lower()

keywords = [
    "volksverhetzung", "hitlergruß", "hakenkreuz",
    "nazi", "rechtsextremistisch", "fremdenfeindlich",
    "nationalsozialismus", "nationalsozialistisch",
    "rassismus", "antisemitismus", "homophobie",
    "transphobie", "sieg heil"
]
action_terms = [
    "graffiti", "angriff", "schlagen", "treten", "schubsen",
    "brandanschlag", "beleidigung", "versammlung", "online posts",
    "raubüberfall", "diebstahl", "körperverletzung",
    "tötungsversuch"
]

time_regex = re.compile(r"\b([0-2]?\d[:\.]?[0-5]?\d)\s*uhr")
date_regex = re.compile(r"\b(\d{1,2}[\./]\d{1,2}[\./]\d{2,4})\b")
age_regex = re.compile(r"\b(\d{1,3})(?:[- ]?jährig(?:e[rn]?)?|\sjahre alt)\b")
gender_regex = re.compile(r"\b(mann|frau|jugendlicher|jugendliche|mädchen|junge)\b")
street_regex = re.compile(r"\b[a-zäöüß]+(?:straße|platz|allee|ring)\b")

diff_threshold = 0.85

for col in [
    'RightWingRelated', 'KeywordMatch', 'ExtractedDate', 'ExtractedTime',
    'ExtractedAge', 'ExtractedGender', 'ExtractedAction', 'KeywordExtracted'
]:
    df[col] = None

def find_keywords(text, terms, threshold):
    tokens = set(re.findall(r"\w+", text))
    hits = []
    for term in terms:
        low = term.lower()
        if re.search(rf"\b{re.escape(low)}\b", text):
            hits.append(term)
        else:
            for tok in tokens:
                if abs(len(tok) - len(low)) <= 3 and difflib.SequenceMatcher(None, tok, low).ratio() >= threshold:
                    hits.append(term)
                    break
    return hits

def find_keywords_with_matches(text, terms, threshold):
    tokens = set(re.findall(r"\w+", text))
    hits = []
    for term in terms:
        low = term.lower()
        for m in re.finditer(rf"\b{re.escape(low)}\b", text):
            hits.append(m.group(0))
        for tok in tokens:
            if abs(len(tok) - len(low)) <= 3 and difflib.SequenceMatcher(None, tok, low).ratio() >= threshold:
                for m in re.finditer(rf"\b{re.escape(tok)}\b", text):
                    hits.append(m.group(0))
    return list(set(hits)) 

# df['KeywordExtracted'] = None

for idx, text in df_text.items():
    kws = find_keywords(text, keywords, diff_threshold)
    related = bool(kws)
    df.at[idx, 'RightWingRelated'] = related
    df.at[idx, 'KeywordMatch'] = kws
    df.at[idx, 'KeywordExtracted'] = find_keywords_with_matches(text, keywords, diff_threshold)


    if related:
        date_match = date_regex.search(str(df.at[idx, 'Date']).lower())
        df.at[idx, 'ExtractedDate'] = date_match.group(1) if date_match else df.at[idx, 'Date']

        time_match = time_regex.findall(text)
        df.at[idx, 'ExtractedTime'] = time_match
        
        df.at[idx, 'ExtractedAge'] = age_regex.findall(text)
        df.at[idx, 'ExtractedGender'] = gender_regex.findall(text)
        df.at[idx, 'ExtractedAction'] = find_keywords(text, action_terms, diff_threshold)

out_path = os.path.join(output_dir, "merged_parsed_documents.csv")
parsed = df[df['RightWingRelated'] == True]
parsed.to_csv(out_path, index=False)
print(f"Saved parsed data to: {out_path}")

Saved parsed data to: output/merged_parsed_documents.csv


## LLM

In [ ]:
import pandas as pd
import glob
import os
import json
import re
import ollama

input_dir = "data"
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

csv_files = glob.glob(os.path.join(input_dir, "*.csv"))
if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {input_dir}")

dfs = []
for file in csv_files:
    df_temp = pd.read_csv(file)
    df_temp["SourceFile"] = os.path.basename(file)
    dfs.append(df_temp)

df = pd.concat(dfs, ignore_index=True)

keywords = ["Volksverhetzung", "Hitlergruß", "Hakenkreuz", "Rechtsextremistisch", "Fremdenfeindlich"]
offence = "Sachbeschädigung, Körperverletzung, Ehrverletzung, Äußerungsdelikt, Totschlag, Vermögensdelikt, Brandstiftung, Verbreitungsdelikt, Gefährdungsdelikt, sonstige"
action = "Hakenkreuz, Hitlergruß, Graffiti, Angriff, Schlagen, Schubsen, Verwendung verfassungswidriger Symbole, Beleidigung, Äußerung, Volksverhetzung, Nazi Parole, Schmiererei, Brandanschlag, Belästigung, Bedrohung, Raubüberfall, Diebstahl, Körperverletzung, Tötungsversuch, Anspucken, Geräusch, Versammlung, Online posts, sonstige"
motive = "Verleugnung des Holocaust, Rassismus, Antisemitismus, Fremdenfeindlichkeit, Homophobie, Transphobie, Anti-Islamismus, Rechtsextremismus, Nationalsozialismus, sonstige, unbestimmt"

offence_prompt = (
    "You are analyzing a police report that contains clear signs of hate or extremist content.\n"
    "Return a valid JSON object with the following fields:\n"
    f"- OffenceType: a list of terms from this set: {offence}\n"
    f"- Action: a list of terms from this set: {action}\n"
    f"- PossibleMotive: a list of terms from this set: {motive}\n\n"
    "Return empty lists if no valid information is found. Respond with JSON only. No explanation or extra text."
)

df["RightWingRelated"] = False
df["KeywordMatch"] = None
df["OffenceType"] = [[] for _ in range(len(df))]
df["Action"] = [[] for _ in range(len(df))]
df["PossibleMotive"] = [[] for _ in range(len(df))]

for i, row in df.iterrows():
    keyword_prompt = (
        f"Below is a document:\n\n"
        f"Title: {row['Title']}\n\n"
        f"Text: {row['Text']}\n\n"
        "Question: Does this document mention any of the following terms: "
        f"{', '.join(keywords)}?\n"
        "If yes, reply with only the term that is mentioned; if not, reply with 'None'."
    )
    keyword_hit = ollama.generate(model="gemma3", prompt=keyword_prompt)["response"].strip()
    if keyword_hit in keywords:
        df.at[i, "RightWingRelated"] = True
        df.at[i, "KeywordMatch"] = keyword_hit

        tagging_prompt = (
            f"Below is a document:\n\n"
            f"Title: {row['Title']}\n\n"
            f"Text: {row['Text']}\n\n"
            f"{offence_prompt}"
        )
        tagging_response = ollama.generate(model="gemma3", prompt=tagging_prompt)["response"].strip()
        try:
            result = json.loads(tagging_response)
        except json.JSONDecodeError:
            match = re.search(r"\{.*\}", tagging_response, re.DOTALL)
            if match:
                try:
                    result = json.loads(match.group(0))
                except json.JSONDecodeError:
                    result = {"OffenceType": [], "Action": [], "PossibleMotive": []}
            else:
                result = {"OffenceType": [], "Action": [], "PossibleMotive": []}

        df.at[i, "OffenceType"] = result.get("OffenceType", [])
        df.at[i, "Action"] = result.get("Action", [])
        df.at[i, "PossibleMotive"] = result.get("PossibleMotive", [])

output_path = os.path.join(output_dir, "tagged_documents.csv")
df.to_csv(output_path, index=False)
print(f"Saved merged tagged documents to: {output_path}")

Saved merged keyword-tagged documents to: output/merged_tagged_documents.csv


### Locations

In [ ]:
import re
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="precise-street-geo")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

prompt_template = """
Extract the most specific geocodable location from the following police report.

Return a single string like:
"Street name, Town, District"
Do not include quotes, explanations, or extra punctuation.
Only return a single clean, geocodable string.
---

Text:
{incident}

Location:
{location}

Locations:
"""

right_wing_df = df[df["RightWingRelated"] == True].copy()
results = []

df["Latitude"] = None
df["Longitude"] = None

for i, row in right_wing_df.iterrows():
    prompt = prompt_template.format(
        incident=row['Text'] or "",
        location=row['Location'] or ""
    )


    response = get_response_from_model(prompt, model).strip()
    response = re.sub(r"^```(?:json|python)?\s*", "", response, flags=re.IGNORECASE)
    response = re.sub(r"\s*```$", "", response, flags=re.IGNORECASE)
    response = re.sub(r"^python\s*", "", response, flags=re.IGNORECASE)

    query1 = f"{response}, Berlin, Germany"
    query2 = f"{row['Location']}, Berlin, Germany" if pd.notna(row["Location"]) else ""
    query3 = ""

    if pd.notna(row["Location"]) and "," in row["Location"]:
        query3 = row["Location"].split(",")[-1].strip() + ", Berlin, Germany"

    location = None


    try:
        location = geocode(query1)
    except Exception as e:
        print(f"⚠️ Error geocoding primary for row {i}: {e}")


    if not location and query2:
        try:
            location = geocode(query2)
        except Exception as e:
            print(f"⚠️ Error geocoding fallback for row {i}: {e}")


    if not location and query3:
        try:
            location = geocode(query3)
        except Exception as e:
            print(f"⚠️ Error geocoding last fallback for row {i}: {e}")


    if location:
        print(f"📍 Row {i}: {location.address} → {location.latitude}, {location.longitude}")
        results.append({
            "Query": response,
            "Latitude": location.latitude,
            "Longitude": location.longitude,
            "TextExcerpt": row["Text"],
            "URL": row["URL"]
        })
        df.at[i, "Latitude"] = location.latitude
        df.at[i, "Longitude"] = location.longitude
    else:
        print(f"⚠️ Geocoding failed for all fallbacks on row {i}: {query1} / {query2} / {query3}")

# geo_df = pd.DataFrame(results)
# geo_df.to_csv("data/berlin_geocoded.csv", index=False)

df.to_csv("data/berlin_geocoded.csv", index=False)

df.head()
